In [1]:
%matplotlib tk

import pyxem as px
from diffsims import generators
import tensorflow as tf
import diffpy.structure
from matplotlib import pyplot as plt
#matplotlib.use("Agg")
import numpy as np
import os
import hyperspy as hs
from tqdm import tqdm

os.chdir('H:/phaseMapping_SPED')
from phasemappingANN import preprocess_for_ANN as pp
from phasemappingANN import training_data_functions as tdf

In [2]:
from keras.models import load_model

In [3]:
directory = r'H:\PhD\CNN\Emil_2xxx_data\SPED_512x512\models'.replace('\\','\\\\')
model = load_model(directory + r'/model_final_2023')

## Load and preprocess experimental data, load tensorflow model

In [2]:
os.chdir('E:\Elisabeth\Al-Cu-Li-Emil\Data')
signal = hs.io.load('SPED_512x512x12_10x10_4p63x4p63_1deg_100Hz_CL12cm_NBD_alpha5_spot1p3_FINAL_rotated_minus60p35.hspy')

In [4]:
rot_angle = -60.35
mask_size = 11
shift = 0.024
nx, ny, kx, ky = signal.axes_manager[0].size, signal.axes_manager[1].size, signal.axes_manager[2].size, signal.axes_manager[3].size
signal_masked = pp.preprocess_NN(signal, rot_angle, mask_size, shift)

Rotating data..
[########################################] | 100% Completed | 103.39 s
Scaling the intensity..


100%|████████████████████████████████████████████████████████████████████████| 262144/262144 [00:45<00:00, 5737.91it/s]


Normalizing between 0 and 1..


100%|███████████████████████████████████████████████████████████████████████| 262144/262144 [00:12<00:00, 20281.51it/s]


In [3]:
directory = r'H:\PhD\CNN\Emil_2xxx_data\SPED_512x512\models'.replace('\\','\\\\')
model = tf.keras.models.load_model(directory + r'/model_final_2023')

### Predict

In [7]:
nx, ny = signal.data.shape[0], signal.data.shape[1]
kx, ky = signal.data.shape[2], signal.data.shape[3]

predictions = model.predict(signal_masked)
signal_prediction = np.zeros((nx*ny))
for i in tqdm(range(nx*ny)):
    signal_prediction[i] = np.argmax(predictions[i])
signal_prediction = signal_prediction.reshape(nx, ny)
plt.close('all')
plt.imshow(signal_prediction)

8192/8192 [==============================] - 937s 114ms/step


100%|██████████████████████████████████████████████████████████████████████| 262144/262144 [00:01<00:00, 211147.23it/s]


In [62]:
phase_map_ANN = hs.signals.Signal2D(signal_prediction)
phase_map_ANN.save(r'H:\PhD\CNN\Emil_2xxx_data\SPED_512x512\Final\Phasemaps\DatasetA\datasetA_phase_map_ANN.hdf5')

### Optionally: Test values for log-shift:

In [ ]:
shift_array = np.arange(0.02 , 0.04, 0.002)
mask_size = 11
errors = np.zeros_like(shift_array)
ground_truth = hs.io.load(r'H:\Ground truths\New-ground-truth\ground_truth_all.hdf5')

for ind, shift in enumerate(shift_array):
    signal_masked = pp.preprocess_NN(signal, rot_angle, mask_size, shift)
    print('predicting..')
    predictions = model.predict(signal_masked, batch_size=2048)
    signal_prediction = np.zeros((nx*ny))
    print('creating phase map')
    for pixel in tqdm(range(nx*ny)):
        signal_prediction[pixel] = np.argmax(predictions[pixel])
    signal_prediction = signal_prediction.reshape(nx, ny)
    phase_map_ANN = hs.signals.Signal2D(signal_prediction)
    error = np.count_nonzero(np.abs((phase_map_ANN-ground_truth).data))/(512*512)
    errors[ind] = error
    print('Error: {}% when shift = {}'.format(error*100, shift))
    del signal_masked
    gc.collect()

limit = 0.00001
index = np.ma.MaskedArray(errors, errors < limit)
index = np.ma.argmin(index)
print('Lowest error: {}% when shift = {}'.format(errors[index]*100, shift_array[index]))